In [ ]:
import os
import shutil
import cv2
import numpy as np
import pathlib
from pathlib import Path
import zipfile
import tarfile

## Training data preparation

In [ ]:
# Set the folder path for FMD rawdata
path = pathlib.Path(os.getcwd())
FMD_rawdata = path/'FMD_rawdata'

In [ ]:
# Ensure you have the required permissions to read/write in the directories you specify when using this function
def unpack_FMD_zip(parent_zip_path, extract_to):
    #Unpacks multiple zip files from a parent zip file.
    
    # Make sure the destination directory exists
    os.makedirs(extract_to, exist_ok=True)
    
    with zipfile.ZipFile(parent_zip_path, 'r') as parent_zip:
        # Extract all zip files from the parent zip file
        for file in parent_zip.namelist():
            if file.endswith('.tar'):
                parent_zip.extract(file, extract_to)
                tarfile_path = os.path.join(extract_to,file)
                # Now extract the zip file just extracted
                with tarfile.open(tarfile_path, 'r') as tar_ref:
                    tar_ref.extractall(extract_to)
                # Optionally delete the inner zip file after extraction
                os.remove(os.path.join(extract_to, file))

In [ ]:
unpack_FMD_zip(FMD_rawdata/'24744648.zip',FMD_rawdata) #24744648 is the zip file name of the FMD dataset when downloading

In [ ]:
# remove # from the line below to delete the original zip file of the FMD dataset
# os.remove(FMD_rawdata/'24744648.zip')# rename folder

In [ ]:
# rename the test folder to fit repository data structure
os.rename(FMD_rawdata/'test_mix',FMD_rawdata/'FMD_testmix')
# move testfolder to fit repository data structure
shutil.move(FMD_rawdata/'FMD_testmix',path)
# update path
FMD_testmix = path/'FMD_testmix'

In [ ]:
DataType = ['avg2','avg4','avg8','avg16','raw','gt']
Sample = ['Confocal_BPAE_B', 'Confocal_BPAE_G', 'Confocal_BPAE_R','Confocal_FISH','Confocal_MICE',
         'TwoPhoton_BPAE_B', 'TwoPhoton_BPAE_G', 'TwoPhoton_BPAE_R', 'TwoPhoton_MICE',
         'WideField_BPAE_B', 'WideField_BPAE_G', 'WideField_BPAE_R']

In [ ]:
scaled = np.zeros((6,512,512,50)) # allocate space to load all DataTypes of a Sample

for i in Sample:
    base = os.path.join(FMD_rawdata,i)
    for j in range(1,21):                # number of FOVs acquired per sample
        files_avg2 = sorted(os.listdir(os.path.join(base, DataType[0],str(j)))) # avg2 directory
        files_avg4 = sorted(os.listdir(os.path.join(base, DataType[1],str(j)))) # avg4 directory
        files_avg8 = sorted(os.listdir(os.path.join(base, DataType[2],str(j)))) # avg8 directory
        files_avg16 = sorted(os.listdir(os.path.join(base, DataType[3],str(j)))) # avg16 directory
        files_avg1 = sorted(os.listdir(os.path.join(base, DataType[4],str(j)))) # raw directory
        files_gt = sorted(os.listdir(os.path.join(base, DataType[5],str(j)))) # gt directory

        for k in range(50):              # number of images acquired per sample per FOV
            if k < len(files_avg2):
                scaled[0, :, :, k] = cv2.imread(os.path.join(base, DataType[0], str(j), files_avg2[k]),cv2.IMREAD_GRAYSCALE)  # avg2
            if k < len(files_avg4):
                scaled[1, :, :, k] = cv2.imread(os.path.join(base, DataType[1], str(j), files_avg4[k]),cv2.IMREAD_GRAYSCALE)  # avg4
            if k < len(files_avg8):
                scaled[2, :, :, k] = cv2.imread(os.path.join(base, DataType[2], str(j), files_avg8[k]),cv2.IMREAD_GRAYSCALE)  # avg8
            if k < len(files_avg16):
                scaled[3, :, :, k] = cv2.imread(os.path.join(base, DataType[3], str(j), files_avg16[k]),cv2.IMREAD_GRAYSCALE)  # avg16
            if k < len(files_avg1):
                scaled[4, :, :, k] = cv2.imread(os.path.join(base, DataType[4], str(j), files_avg1[k]),cv2.IMREAD_GRAYSCALE)  # raw
            if len(files_gt) > 0:
                scaled[5, :, :, k] = cv2.imread(os.path.join(base, DataType[5], str(j), files_gt[0]),cv2.IMREAD_GRAYSCALE)  # gt / avg50

        # Create directories for saved images
        noisy_dir = os.path.join(path, 'FMD_noisy')
        if not os.path.exists(noisy_dir):
            os.makedirs(noisy_dir)
            
        gt_dir = os.path.join(path, 'FMD_GT')
        if not os.path.exists(gt_dir):
            os.makedirs(gt_dir)

        # save images in respective directories
        for l in range(5):             
            for k in range(50):
                filename = f"{i}_{DataType[l]}_FOV{j}_{k + 1}.png"
                cv2.imwrite(os.path.join(noisy_dir, filename), scaled[l, :, :, k]) # avg1, avg2, avg4, avg8, avg16
                cv2.imwrite(os.path.join(gt_dir, filename), scaled[5, :, :, k]) # corresponding ground truth pngs
        
        # clear scaled
        scaled.fill(0)  # Reset the scaled array

## Test data preparation

In [ ]:
def split_image_into_tiles(image, tile_size):    
    #Splits an image into tiles of specified size.
    
    tiles = []
    height, width, _ = image.shape
    
    for y in range(0, height, tile_size):
        for x in range(0, width, tile_size):
            tile = image[y:y + tile_size, x:x + tile_size]
            # Check if the tile is the right size (to avoid edge tiles)
            if tile.shape[0] == tile_size and tile.shape[1] == tile_size:
                tiles.append(tile)
    
    return tiles

In [ ]:
def create_overlapping_patches(image, patch_size, overlap):
    img_array = np.array(image)
    patches = []
    coords = []

    step = patch_size - overlap
    height, width = img_array.shape[:2]
    
    for y in range(0, height - patch_size + 1, step):
        for x in range(0, width - patch_size + 1, step):
            patch = img_array[y:y + patch_size, x:x + patch_size]
            coords.append((x,y))
            patches.append(patch)
    return patches

In [ ]:
def process_images_in_folder(folder_path, tile_size, tag):
    #Processes all images in the specified folder, splitting them into tiles.
    
    # Create output folder for tiles
    output_folder = Path(folder_path)
    output_folder = os.path.join(output_folder.parent.absolute(),'noisy_tiles')
    os.makedirs(output_folder, exist_ok=True)

    # Loop through all files in the folder
    for i,filename in enumerate(os.listdir(folder_path)):
        if filename.lower().endswith('.png'):
            image_path = os.path.join(folder_path, filename)
            image = cv2.imread(image_path)

            if image is None:
                print(f"Unable to read image: {image_path}")
                continue
            
            print(f"Processing image({i}) {filename}")
            #tiles = split_image_into_tiles(image, tile_size)
            tiles = create_overlapping_patches(image, tile_size, 128)
            
            # Save each tile
            for k, tile in enumerate(tiles):
                tile_filename = f"{os.path.splitext(filename)[0]}{tag}_tile_{k}.png"
                tile_path = os.path.join(output_folder, tile_filename)
                cv2.imwrite(tile_path, tile)

In [ ]:
# Set the folder path to FMD testset and the tile size
tile_size = 256

In [ ]:
# Process images of various noise levels in FMD folder
noise_levels = ['raw','avg2','avg4','avg8','avg16']
for i, levels in enumerate(noise_levels):
    print(f'working on {levels} images')
    process_images_in_folder(FMD_testmix/levels,tile_size,levels)